## 1. 필요한 모듈을 불러온다.

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import tensorflow as tf
import numpy as np
import pandas as pd
from pprint import pprint
from cus_tool_copy import *
from tqdm import tqdm

tf.reset_default_graph() # 텐서플로우 그래프를 초기화 한다.

/Users/ku/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 2. corpus를 불러오고 차원을 확인 해 본다.

In [2]:
corpus = pd.read_table("/Users/ku/Desktop/raw.csv", delimiter=",").values
pprint(corpus)
pprint(corpus.shape)

array([['dallas take off', ' plane take off from dallas'],
       ['dallas land', ' plane land at dallas'],
       ['chicago take off', ' plane take off from chicago'],
       ['chicago land', ' plane land at chicago'],
       ['los angeles take off', ' plane take off from los angeles'],
       ['los angeles land', ' plane land at los angeles'],
       ['new york take off', ' plane take off from new york'],
       ['new york land', ' plane land at new york'],
       ['miami take off', ' plane take off from miami'],
       ['miami land', ' plane land at miami']], dtype=object)
(10, 2)


## 3. lookup table를 생성하고 이를 이용하여 데이터를 인덱스로 맵핑한다.

In [3]:
idx2word, word2idx, vocab_size = make_vocab(corpus)
temp_encoder_data, temp_decoder_data, temp_targets_data = make_temp_data(corpus,idx2word=idx2word, word2idx=word2idx)
print('vocab_size: {} \n'.format(vocab_size))
print('idx2word: \n {} \n'.format(idx2word))
print('word2idx: \n {} \n'.format(word2idx))

print('temp_encoder_data: \n {} \n'.format(temp_encoder_data))
print('temp_decoder_data: \n {} \n'.format(temp_decoder_data))
print('temp_targetsr_data: \n {} \n'.format(temp_targets_data))

vocab_size: 17 

idx2word: 
 {0: 'at', 1: 'off', 2: 'take', 3: 'miami', 4: 'plane', 5: 'york', 6: 'new', 7: 'land', 8: 'angeles', 9: 'chicago', 10: 'from', 11: 'los', 12: 'dallas', 13: '<start>', 14: '<end>', 15: '<pad>', 16: '<unk>'} 

word2idx: 
 {'at': 0, 'off': 1, 'take': 2, 'miami': 3, 'plane': 4, 'york': 5, 'new': 6, 'land': 7, 'angeles': 8, 'chicago': 9, 'from': 10, 'los': 11, 'dallas': 12, '<start>': 13, '<end>': 14, '<pad>': 15, '<unk>': 16} 

temp_encoder_data: 
 [[12, 2, 1], [12, 7], [9, 2, 1], [9, 7], [11, 8, 2, 1], [11, 8, 7], [6, 5, 2, 1], [6, 5, 7], [3, 2, 1], [3, 7]] 

temp_decoder_data: 
 [[4, 2, 1, 10, 12], [4, 7, 0, 12], [4, 2, 1, 10, 9], [4, 7, 0, 9], [4, 2, 1, 10, 11, 8], [4, 7, 0, 11, 8], [4, 2, 1, 10, 6, 5], [4, 7, 0, 6, 5], [4, 2, 1, 10, 3], [4, 7, 0, 3]] 

temp_targetsr_data: 
 [[4, 2, 1, 10, 12], [4, 7, 0, 12], [4, 2, 1, 10, 9], [4, 7, 0, 9], [4, 2, 1, 10, 11, 8], [4, 7, 0, 11, 8], [4, 2, 1, 10, 6, 5], [4, 7, 0, 6, 5], [4, 2, 1, 10, 3], [4, 7, 0, 3]] 



## 4. 각 데이터에 pad, end 토큰을 삽입하여 최종적인 데이터를 만든다.

In [4]:
encoder_data, decoder_data, targets_data, max_decoder_steps = insert_tokens(temp_encoder_data, temp_decoder_data, temp_targets_data, word2idx)
data_checker(encoder_data, idx2word)
data_checker(decoder_data, idx2word)
data_checker(targets_data, idx2word)

dallas take off <pad>
dallas land <pad> <pad>
chicago take off <pad>
chicago land <pad> <pad>
los angeles take off
los angeles land <pad>
new york take off
new york land <pad>
miami take off <pad>
miami land <pad> <pad>
<start> plane take off from dallas <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<start> plane land at dallas <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<start> plane take off from chicago <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<start> plane land at chicago <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<start> plane take off from los angeles <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<start> plane land at los angeles <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<start> plane take off from new york <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<start> plane land at new york <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<start> plane take off from miami <pad> <pad> <pad> <pad> <pad> <pad> <pad> 

## 5. mask를 사용하기 위해서 sequence_length를 구한다.

In [5]:
encoder_sequence_length = sequence_length_maker(encoder_data, word2idx)
decoder_sequence_length = sequence_length_maker(decoder_data, word2idx)
print(encoder_sequence_length)
print(decoder_sequence_length)

[3, 2, 3, 2, 4, 3, 4, 3, 3, 2]
[6, 5, 6, 5, 7, 6, 7, 6, 6, 5]


## 6. 하이퍼파라미터를 정의한다.

버퍼사이즈 레퍼런스
https://stackoverflow.com/questions/46444018/meaning-of-buffer-size-in-dataset-map-dataset-prefetch-and-dataset-shuffle

In [6]:
## 3. 하이퍼파라미터를 설정한다.
class_size = vocab_size
batch_size = 3
lr = 0.01
hidden_size = 128
epochs = 100
save_per_ckpt = 10
num_layer = 1 # stacked RNN을 위한 하이퍼파라미터. 몇개의 레이어를 쌓을지 정해준다.

## 7. 모델을 클래스로 정의한다.

In [7]:
## seq2seq 모델을 만든다.

class Object2Text:
    '''
    Arguments:
        btc_enc_seq_len: placeholder([None,]), 인코더 부분의 패딩을 제외한 실제 의미를 가지는 토큰의 개수를 배치별로 나타냄.
        btc_enc_inp: placeholder([None, None]), 인코더의 인풋(batch size, time steps)
        btc_dec_seq_len: placeholder([None,]), 디코더 부분의 패딩을 제외한 실제 의미를 가지는 토큰의 개수를 배치별로 나타냄.
        btc_dec_inp: placeholder([Noen, None]), 디코더의 인풋(batch size, time steps)
        btc_targets: placeholder([None, None]), groud truth(batch size, time steps)
        word2idx: dictionary, '단어'를 키 값으로 조회하면 그에 해당하는 인덱스를 돌려주는 딕셔너리.
        idx2word: dictionary, '인덱스'를 키 값으로 조회하면 그에 해당하는 인덱스를 돌려주는 딕셔너리.
        vocab_size: scalar, 단어사전의 크기(단어사전에 있는 단어의 개수)
        class_size: scalar, 디코더의 아웃풋을 fully connected 레이어와 연결하여 최종적으로 몇개의 클래스로 나타낼지 결정. 일반적으로 vocab size와 같게 하면 된다.
        hidden_size: scalar, seq2seq 모델의 히든 유닛의 개수
    '''
    
    def __init__(self, btc_enc_seq_len, btc_enc_inp, btc_dec_seq_len, btc_dec_inp, btc_targets, word2idx=word2idx, idx2word=idx2word, 
                     vocab_size=vocab_size, class_size=class_size, hidden_size=hidden_size, num_layer=num_layer, max_decoder_steps=max_decoder_steps):
        
        # 생성자의 인자들을 클래스멤버(?)로 만들어준다.
        self._enc_seq_len = btc_enc_seq_len
        self._enc_inp = btc_enc_inp
        self._dec_seq_len = btc_dec_seq_len
        self._dec_inp = btc_dec_inp
        self._targets = btc_targets
        self._word2idx = word2idx
        self._idx2word = idx2word
        self._vocab_size = vocab_size
        self._class_size = class_size
        self._hidden_size = hidden_size
        self._num_layer = num_layer
        self._max_decoder_steps = max_decoder_steps
    
        # 1) encoder의 인풋값을 ont-hot 인코딩(임베딩) 한다.
        with tf.variable_scope('enc_embed_layer'):
            encoder_oh = tf.one_hot(indices=self._enc_inp, depth=self._vocab_size) # encoder_oh: tensor, (batch_size, time steps, vocab_size)

        #  2) encoder 셀을 쌓는다.
        with tf.variable_scope('encoder'):
            enc_cell = tf.contrib.rnn.BasicLSTMCell(num_units=self._hidden_size, activation=tf.nn.tanh)
            enc_cell = tf.nn.rnn_cell.MultiRNNCell([enc_cell] * self._num_layer)
            _outputs, enc_states = tf.nn.dynamic_rnn(cell=enc_cell, inputs=encoder_oh, sequence_length=self._enc_seq_len, dtype=tf.float32)

        # 3) 디코더의 임베딩을 원-핫인코딩으로 한다. tf.one_hot 함수를 사용해서 원-핫 인코딩을 해도 되지만. 나중에 GreedyEmbeddingsHelper를 사용하기 위해서는 임베딩 매트릭스를 만들어 주어야 한다.
        with tf.variable_scope('dec_embed_layer'):
            decoder_eye = tf.eye(num_rows=self._vocab_size)
            decoder_embeddings = tf.get_variable(name='dec_embed', initializer=decoder_eye, trainable=False) # tf.nn.embedding_lookup 함수를 사용하려면 tf.get_variable을 params의 값으로

            #decoder_embeddings = tf.get_variable(name='dec_embed', initializer=decoder_eye, trainable=False) # tf.nn.embedding_lookup 함수를 사용하려면 tf.get_variable을 params의 값으로
            decoder_oh = tf.nn.embedding_lookup(params=decoder_embeddings, ids=self._dec_inp)    

        # 4) decoder 셀을 쌓는다.    
        with tf.variable_scope('decoder'):
            
            # decoder cell for both training and inference(학습과 추론과정은 같은 '가중치'를 공유함.)
            dec_cell = tf.contrib.rnn.BasicLSTMCell(self._hidden_size, activation=tf.nn.tanh)
            dec_cell = tf.nn.rnn_cell.MultiRNNCell([dec_cell] * self._num_layer)
            
            # output projection (replacing `OutputProjectionWrapper`)
            output_layer = tf.layers.Dense(vocab_size)
#             score_cell = tf.contrib.rnn.OutputProjectionWrapper(cell=dec_cell, output_size=self._class_size)

            # 4-1) decoder의 training과 관련된 셀을 쌓는다.
            with tf.variable_scope('training'):
                train_helper = tf.contrib.seq2seq.TrainingHelper(inputs=decoder_oh, sequence_length=self._dec_seq_len, time_major=False)
                train_decoder = tf.contrib.seq2seq.BasicDecoder(cell=dec_cell, helper=train_helper, initial_state=enc_states, output_layer=output_layer)
                
                # self._train_outputs.rnn_output: [batch_size x max(self._max_decoder_steps) x vocab_size]
                # => 각 배치에서 가장 '긴' 값을 2번째 차원의 값으로 가진다는 점을 주의!! seq2seq2 loss 함수를 사용할 때 shape에 주의해야 한다.
                # => bucketing과 연관이 있는 것 같은데.... 확실하지는 않다...
                self._train_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder=train_decoder, impute_finished=True, output_time_major=False, 
                                                                                                                maximum_iterations=self._max_decoder_steps)
                
                # logits: [batch_size x max_dec_step x dec_vocab_size]
                # => tf.identity: seq2seq 함수에서 logits에 들어갈 값 선언. tf.identity는 tensor의 contents와 shape를 똑같이 copy한다.
                logits = tf.identity(self._train_outputs.rnn_output, name='logits')
                
                # btc_max_len: => logits의 shape에 맞춰주기 위함!!
                # => decoder input "배치" 중에서  최대 sequence length(의미 있는 인풋이 들어오는 time step)
                btc_max_len = tf.reduce_max(self._dec_seq_len, name='max_dec_len')
                
                # targets: 
                targets = tf.slice(input_=self._targets, begin=[0, 0], size=[-1, btc_max_len], name='targets')
                              

            # 4-2) decoder의 inference와 관련된 셀을 쌓는다.
            with tf.variable_scope('inference'):
                # 모델의 input을 dynamic shape로 하였으므로.. tf.shape(btc_enc_seq_len)로 해야 한다. btc_enc_seq_len.shape는 static shape일 때 사용 하는 것이다.
                start_tokens = tf.fill(dims=tf.shape(btc_enc_seq_len), value=self._word2idx['<start>'])

                infer_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embedding=decoder_embeddings, 
                                                                                                              start_tokens=start_tokens, 
                                                                                                              end_token=self._word2idx['<end>'])
                infer_decoder = tf.contrib.seq2seq.BasicDecoder(cell=dec_cell, helper=infer_helper, initial_state=enc_states, output_layer=output_layer)
                self._infer_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(infer_decoder, impute_finished=True, maximum_iterations=self._max_decoder_steps)
        
        with tf.variable_scope('seq2seq_loss'):
            #masksing: [batch_size x max_dec_len]
            #=> ignore outputs after `dec_senquence_length+1` when calculating loss
            masking = tf.sequence_mask(lengths=self._dec_seq_len, maxlen=btc_max_len, dtype=tf.float32)
                
            #internal: `tf.nn.sparse_softmax_cross_entropy_with_logits`
            self.seq2seq_loss = tf.contrib.seq2seq.sequence_loss(logits=logits, targets=targets, weights=masking)
            
    def infer(self, sess,  btc_enc_seq_len, btc_enc_inp):
        feed_infer = {self._enc_seq_len: btc_enc_seq_len, self._enc_inp: btc_enc_inp}
        return sess.run(self._infer_outputs.sample_id, feed_dict=feed_infer)            

## 8. 배치 관련 설정을 한다.

In [8]:
## 인풋, 타겟, sequence_length와 관련된 placeholder를 정의한다.
encoder_input = tf.placeholder(dtype=tf.int32, shape=[None, None]) # (batch size, time steps)
decoder_input = tf.placeholder(tf.int32, [None, None]) # (batch size, time steps)
targets = tf.placeholder(tf.int32, [None, None]) # (batch size, time steps)
encoder_seq_len = tf.placeholder(tf.int32, [None]) # (batch size, ) 인코더 배치에 대응하는 sequence_length
decoder_seq_len = tf.placeholder(tf.int32, [None]) # (batch size,) 디코더 배치에 대응하는 sequence_length

## 배치 iterator를 정의하여 학습시 사용할 준비를 한다.
dataset = tf.data.Dataset.from_tensor_slices((encoder_seq_len, encoder_input, decoder_seq_len, decoder_input, targets)) # 여러 플레이스 홀더를 데이터셋으로 하고
dataset = dataset.shuffle(10000).batch(batch_size) # 랜덤으로 데이터셋을 섞은 다음에 배치크기는 미리 정의된 만큼 할 생각이야.
iterator = dataset.make_initializable_iterator() # 위와 같은 사항을 반영하여 전체 데이터셋에서 배치 크기만큼 반복적으로 꺼내올 수 있는 iterator를 만들어 줘.
btc_enc_seq_len, btc_enc_inp, btc_dec_seq_len, btc_dec_inp, btc_targets  = iterator.get_next() # 배치를 꺼내오는 명령문은 next_batch로 할게. (sess.run에서 실행 시켜야 다음 배치를 꺼내온다.)
# next_element = iterator.get_next() # 배치를 꺼내오는 명령문은 next_batch로 할게. (sess.run에서 실행 시켜야 다음 배치를 꺼내온다.)

print(dataset)
# print(next_element)
print(btc_enc_seq_len, btc_enc_inp, btc_dec_seq_len, btc_dec_inp, btc_targets)

<BatchDataset shapes: ((?,), (?, ?), (?,), (?, ?), (?, ?)), types: (tf.int32, tf.int32, tf.int32, tf.int32, tf.int32)>
Tensor("IteratorGetNext:0", shape=(?,), dtype=int32) Tensor("IteratorGetNext:1", shape=(?, ?), dtype=int32) Tensor("IteratorGetNext:2", shape=(?,), dtype=int32) Tensor("IteratorGetNext:3", shape=(?, ?), dtype=int32) Tensor("IteratorGetNext:4", shape=(?, ?), dtype=int32)


## 9. object2text 클래스를 생성한다.

In [9]:
object2text = Object2Text(btc_enc_seq_len, btc_enc_inp, btc_dec_seq_len, btc_dec_inp, btc_targets)

## 10. optimizer를 정의하고 모델을 학습시킨다.

In [10]:
train_op = tf.train.AdamOptimizer(lr).minimize(object2text.seq2seq_loss)

In [11]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
loss_hist=[]

for epoch in tqdm(range(100)):
    
    # iterator.initalizer를 반복문 안에 넣어줘야 배치를 다 꺼내 쓴 다음에 다시 배치를 쓸 수 있다. 이것을 반복문 바깥에 두면 계속 OutOfRangeError 에러가 난다.
    sess.run(iterator.initializer, feed_dict = {encoder_input: encoder_data, decoder_input: decoder_data, 
                                                targets: targets_data, encoder_seq_len: encoder_sequence_length, decoder_seq_len: decoder_sequence_length})
    print("Epoch: %s" %epoch)

    while True:
        try:
            _, loss = sess.run([train_op, object2text.seq2seq_loss])
            print(loss)            
            
        # 에러가 발생한다는 것은 더 이상 꺼내 올 배치가 없다는 것이다. 따라서 배치를 꺼내오는 while문을 종료하고 다음 epoch로 넘어간다.
        except tf.errors.OutOfRangeError:
            break

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 0


  6%|▌         | 6/100 [00:00<00:06, 14.29it/s]

2.8339288
2.7643921
2.6500144
2.4935522
Epoch: 1
2.6229265
2.1464653
2.1503894
2.1246612
Epoch: 2
1.8885605
1.6242279
1.5329415
1.2729223
Epoch: 3
1.0690578
0.99697924
1.022404
0.8140662
Epoch: 4
0.9250781
0.73272216
1.1178355
0.8666382
Epoch: 5
0.9391855
0.6121753
0.6580689
0.6126131
Epoch: 6
0.56777906
0.6083157
0.58363575
1.0477909
Epoch: 7
0.45639148
0.52411234
0.62947613
0.4725864
Epoch: 8
0.3514487
0.5558199
0.4320736
0.7163065
Epoch: 9
0.49528563
0.47470757


 16%|█▌        | 16/100 [00:00<00:03, 25.20it/s]

...

0.0010936446
0.0011081729
0.0009949052
Epoch: 75
0.0010961076
0.0010765356
0.00118565
0.00096453936
Epoch: 76
0.0010498873
0.0010460535
0.0011221233
0.0011433614
Epoch: 77
0.0009173577
0.0010657688
0.0010949433
0.0012075223
Epoch: 78
0.0009745322
0.0009836773
0.0011241664
0.0010270171
Epoch: 79
0.0010568394
0.00087638485
0.0011215593
0.00084699225
Epoch: 80
0.0010691311
0.0010199386
0.00088145334
0.0008900144
Epoch: 81
0.000921458
0.0009457876
0.0010247981
0.00085931853
Epoch: 82
0.0010220786
0.0009321545
0.000800286
0.0010456176
Epoch: 83
0.0009509841
0.0008941538


 91%|█████████ | 91/100 [00:02<00:00, 39.79it/s]

0.0008539539
0.0010036109
Epoch: 84
0.00086984225
0.00097648863
0.0008683368
0.0007597475
Epoch: 85
0.00079881895
0.0008706771
0.0009559212
0.000866524
Epoch: 86
0.00085520744
0.0008999829
0.0008161418
0.00081253017
Epoch: 87
0.0007471296
0.00085817877
0.0009066689
0.00078226265
Epoch: 88
0.0007917214
0.0008824637
0.0007142424
0.0010042212
Epoch: 89
0.0007597052
0.0008604749
0.00076172897
0.0008332303
Epoch: 90
0.0007830239
0.00078306947
0.0007866509
0.00078580424
Epoch: 91
0.00078024564
0.00071686204
0.0008442728
0.0006539693
Epoch: 92
0.0006703952


100%|██████████| 100/100 [00:02<00:00, 40.03it/s]

0.00080716604
0.00076689804
0.0008032552
Epoch: 93
0.0007741097
0.0006886966
0.0007408135
0.00077615195
Epoch: 94
0.0006846683
0.00076431915
0.00075599103
0.00060976954
Epoch: 95
0.0007602642
0.000736194
0.0006313978
0.0007165091
Epoch: 96
0.00066930626
0.000663395
0.00078613276
0.0005801473
Epoch: 97
0.00068885856
0.0006836762
0.0006969871
0.0006243554
Epoch: 98
0.0006617251
0.00070064655
0.00065692246
0.0006652907
Epoch: 99
0.0007345926
0.00066690304
0.00062007667
0.0005414921


## 11. 학습된 가중치가 저장된 sess를 통해서 inference를 한다.

In [13]:
prediction = object2text.infer(sess, btc_enc_inp=encoder_data, btc_enc_seq_len=encoder_sequence_length)

In [14]:
print(prediction)

[[ 4  2  1 10 12 14  0]
 [ 4  7  0 12 14  0  0]
 [ 4  2  1 10  9 14  0]
 [ 4  7  0  9 14  0  0]
 [ 4  2  1 10 11  8 14]
 [ 4  7  0 11  8 14  0]
 [ 4  2  1 10  6  5 14]
 [ 4  7  0  6  5 14  0]
 [ 4  2  1 10  3 14  0]
 [ 4  7  0  3 14  0  0]]


In [18]:
list(map(lambda row: [idx2word[idx] for idx in row], prediction))

[['plane', 'take', 'off', 'from', 'dallas', '<end>', 'at'],
 ['plane', 'land', 'at', 'dallas', '<end>', 'at', 'at'],
 ['plane', 'take', 'off', 'from', 'chicago', '<end>', 'at'],
 ['plane', 'land', 'at', 'chicago', '<end>', 'at', 'at'],
 ['plane', 'take', 'off', 'from', 'los', 'angeles', '<end>'],
 ['plane', 'land', 'at', 'los', 'angeles', '<end>', 'at'],
 ['plane', 'take', 'off', 'from', 'new', 'york', '<end>'],
 ['plane', 'land', 'at', 'new', 'york', '<end>', 'at'],
 ['plane', 'take', 'off', 'from', 'miami', '<end>', 'at'],
 ['plane', 'land', 'at', 'miami', '<end>', 'at', 'at']]